In [1]:
import streamlit as st
import numpy as np
import pandas as pd
from PIL import Image
import gspread
import json
import pandas as pd
#ServiceAccountCredentials：Googleの各サービスへアクセスできるservice変数を生成します。
from oauth2client.service_account import ServiceAccountCredentials

#2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
json_path = 'kitaichi-fd2dd716665b.json'
#認証情報設定
#ダウンロードしたjsonファイル名をクレデンシャル変数に設定（秘密鍵、Pythonファイルから読み込みしやすい位置に置く）
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
#OAuth2の資格情報を使用してGoogle APIにログインします。
gc = gspread.authorize(credentials)
# １．ファイル名を指定してワークブックを選択
workbook = gc.open('収支表')
sh_secchi = workbook.worksheet('設置一覧')
sh_shop = workbook.worksheet('店一覧')
sh_slot_kisyu = workbook.worksheet('スロット機種一覧')
df_secchi = pd.DataFrame(sh_secchi.get_all_values()[1:], columns=sh_secchi.get_all_values()[0])
df_shop = pd.DataFrame(sh_shop.get_all_values()[1:], columns=sh_shop.get_all_values()[0])
df_slot_kisyu = pd.DataFrame(sh_slot_kisyu.get_all_values()[1:], columns=sh_slot_kisyu.get_all_values()[0])

In [2]:
sh_kari = workbook.worksheet('シート7')

In [4]:
df = pd.DataFrame(sh_kari.get_all_values()[1:], columns=sh_kari.get_all_values()[0])

In [9]:
df

,機種番号,カテゴリ,項目,内容
0,s_0096,基本情報,機種名,パチスロこの素晴らしい世界に祝福を!
1,s_0096,基本情報,型名,Ｓこの素晴らしい世界に祝福をＺＲ
2,s_0096,基本情報,メーカー,サミー
3,s_0096,基本情報,タイプ,AT
4,s_0096,基本情報,コイン持ち,36.3
...,...,...,...,...
640,s_0096,天井狙い,591,6154
641,s_0096,天井狙い,592,6181
642,s_0096,天井狙い,593,6209
643,s_0096,天井狙い,594,6236


In [8]:
kisyu_df = df[df['機種番号'] == 's_0096']

In [10]:
tenzyo_df = kisyu_df[kisyu_df['カテゴリ'] == '天井狙い']

In [15]:
tenzyo_df

,機種番号,カテゴリ,項目,内容
49,s_0096,天井狙い,0,-476
50,s_0096,天井狙い,1,-470
51,s_0096,天井狙い,2,-465
52,s_0096,天井狙い,3,-460
53,s_0096,天井狙い,4,-455
...,...,...,...,...
640,s_0096,天井狙い,591,6154
641,s_0096,天井狙い,592,6181
642,s_0096,天井狙い,593,6209
643,s_0096,天井狙い,594,6236


In [24]:
list(tenzyo_df['項目'])[30]

'30'

In [25]:
list(tenzyo_df['内容'])[30]

'31'

In [40]:
int(list(tenzyo_df['内容'])[30])*0.92

28.52

In [37]:
1000/46

21.73913043478261

In [38]:
1000/50

20.0

In [39]:
20/21.7

0.9216589861751152